# Transfer extraction script

Extracts transfers of funds from Chicago Board of Education agendas and exports them as CSV files. Handles some edge cases, but OCR issues can cause some transfers to not be parsed properly.

In [22]:
import json
from collections import Counter

docs_metadata = json.load(open('../data/DocumentCloud/documents.json'))

STOP_WORDS = ['Detroit', 'Cleveland', 'Akron', 'Ohio', 'Allegheny', 'Cuyahoga', 'Summit County', 'Wayne County', 'Michigan' ]
def chicago_filter(doc_json):
    for word in STOP_WORDS:
        if word.lower() in doc_json['source'].lower():
            return False
    return True

# Filter non-Chicago documents
filtered_docs = filter(chicago_filter, docs_metadata)
chicago_docs = [x for x in filtered_docs]

In [23]:
"""Return document information for a given source.

source: search keyword for source name, case insensitive. Empty string searches all sources.
title_keyword: search keyword for document title, case insensitive. Default: ""
only_title: If set to True, only returns title of matching documents, not the entire json as dict. Default: False.
"""
def get_docs_for_source(source, title_keyword="", only_title=False):
    filter_func = lambda x:  source.lower() in x['source'].lower()
    if title_keyword:
        filter_func = lambda x:  source.lower() in x['source'].lower() and title_keyword in x['title'].lower()
    source_filter = filter(filter_func, chicago_docs)
    if not only_title:
        source_docs = [x for x in source_filter]
    else:
        source_docs = [x['title'] for x in source_filter]
    print("Documents found in {}: {}".format(source, len(source_docs)))
    return source_docs

In [24]:
import re

TEXT_DATABASE_PATH = "../data/DocumentCloud/text/"
ACTION_ID = "ACTION ID"
UNFORMATTED_KEY = "UNFORMATTED"


"""Process a single set of transfers. Returns transfers as a list of dictionaries.

action_id: The action id associated with the given set of transfers.
action_text: The text containing the transfers.
"""
def process_transfer_action(action_id, action_text):
    
    # Split text into individual transfers by numbered heading (i.e. "2.", "45.")
    transfers = re.split('^\s*([0-9]+)\.', action_text, flags=re.MULTILINE)
    
    # Create list of pairs (transfer number, transfer text)
    transfer_list = []
    for i in range(1, len(transfers), 2):
        transfer_list.append((transfers[i], transfers[i+1]))
    
    
    transfer_rows = []
    for t in transfer_list:
        transfer_info = {"Number": t[0], "Action": action_id}
        
        # Split transfer into sections.
        sections = t[1].split("\n\n")
        
        # Remove page numbers and whitespace.
        cleaned_sections = [x for x in sections if x.strip() and x.strip() != action_id and not x.isnumeric()]

        
        for i, x in enumerate(cleaned_sections):
            if i == 0:
                # First section is typically the name with "Rationale:" appended to the end
                name = x.replace("\nRationale:", "")
                transfer_info["Name"] = name
            elif i == 1:
                # Second section is typically the rationale for the transfer.
                transfer_info["Rationale"] = x
            else:
                # Amount can be in different sections. If "Amount:" is present, extract dollar amount
                split_x = x.split("Amount:")
                if len(split_x) > 1:
                    transfer_info["Amount"] = split_x[1].strip()
                    x = split_x[0]
                
                # Each line in the transfer details starts with an id number - split by those
                split_section = re.split('^([0-9]+)\s', x, flags=re.MULTILINE)
                
                # First line indicates whether section contains "From" or "To" information
                section_type = ""
                if "Transfer From:" in split_section[0]:
                    section_type = "From"
                elif "Transfer To:" in split_section[0]:
                    section_type = "To"

                if section_type:
                    for j in range(1, len(split_section), 2):
                        # Example headers: "From Line 1", "From Line 2" ... "To Line 1", "To Line 2" ...
                        header = "{} Line {}".format(section_type, int((j+1)/2))
                        transfer_info[header] = (split_section[j] + " " + split_section[j+1]).strip()
                elif x.strip():
                    # Remaining text stored in last column
                    if UNFORMATTED_KEY in transfer_info:
                        transfer_info[UNFORMATTED_KEY] += x
                    else:
                        transfer_info[UNFORMATTED_KEY] = x
                        
        transfer_rows.append(transfer_info)
    return transfer_rows
    
"""Extract transfers from a document.

doc_id: Document id.
"""
def extract_transfers(doc_id):
    # Get document text.
    fname = TEXT_DATABASE_PATH + doc_id + ".txt"
    with open(fname, "r") as f:
        doc_text = f.read()
    
    # Split document into actions. Board of Education actions have the format ##-####
    actions = re.split('(^\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z]\d$)', doc_text, flags=re.MULTILINE)
    action_list = []
    for i in range(1, len(actions), 2):
        action_list.append((actions[i], actions[i+1]))
    
    # Get actions that contain transfers of funds
    transfer_actions = set()
    for a in action_list:
        if "TRANSFER OF FUNDS" in a[1]:
            transfer_actions.add(a[0])
    
    # Process actions with transfers of funds, and add them to a single list of transfers
    transfers = []
    for action_id in transfer_actions:
        action_text = ""
        for a in action_list:
            if a[0] == action_id:
                action_text += a[1]
        transfers += process_transfer_action(action_id, action_text)
    return transfers

In [25]:
import csv
import os

FOLDER_NAME = "transfers"

"""Extracts transfers from agendas and exports them to csv files.

school_agendas: List of BOE agendas as dicts
"""
def export_transfers(school_agendas):
    for agenda in school_agendas:
        agenda_name = agenda["title"]
        print("Document Name: {}".format(agenda_name))
        print("Document URL: {}".format(agenda["canonical_url"]))
        transfers = extract_transfers(agenda["id"])
        transfer_count = len(transfers)
        print("Number of transfers extracted: {}".format(transfer_count))
        key_counts = sum_key_counts(transfers)
        print("Keys: {} \n".format(str(key_counts)))
        if transfer_count > 0:
            export_csv(agenda_name, key_counts, transfers)

"""Count occurances of keys in processed transfers.

transfers: Processed transfers as list of dicts.
"""
def sum_key_counts(transfers):
    key_counts = {}
    for c in transfers:
        for k in c.keys():
            if k in key_counts:
                key_counts[k] += 1
            else:
                key_counts[k] = 1
    return key_counts

"""Export transfers to csv file in subdirectory.

transfers: Processed transfers as list of dicts.
"""
def export_csv(name, key_counts, transfers):
    keys = key_counts.keys()
    if not os.path.exists(FOLDER_NAME):
        os.makedirs(FOLDER_NAME)
    date = re.search(r'\d{4}-\d{2}-\d{2}', name)
    with open("{}/BOE Agenda Transfer of Funds - {} .csv".format(FOLDER_NAME, date.group()), "w") as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(transfers)


In [26]:
school_agendas = get_docs_for_source("Chicago Public Schools", title_keyword="agenda")
export_transfers(school_agendas)

Documents found in Chicago Public Schools: 30
Document Name: Board of Education 2019-12-11 - Agenda For The Board Of Education
Document URL: https://www.documentcloud.org/documents/6572197-Board-of-Education-2019-12-11-Agenda-For-The.html
Number of transfers extracted: 1026
Keys: {'Number': 1026, 'Action': 1026, 'Name': 1026, 'Rationale': 1026, 'From Line 1': 1026, 'From Line 2': 1025, 'From Line 3': 1025, 'From Line 4': 1025, 'From Line 5': 1003, 'To Line 1': 1026, 'To Line 2': 1026, 'To Line 3': 1026, 'To Line 4': 1026, 'To Line 5': 1010, 'Amount': 1026, 'UNFORMATTED': 45} 

Document Name: Board of Education 2019-11-20 - Agenda For The Board Of Education
Document URL: https://www.documentcloud.org/documents/6553016-Board-of-Education-2019-11-20-Agenda-For-The.html
Number of transfers extracted: 4171
Keys: {'Number': 4171, 'Action': 4171, 'Name': 4171, 'Rationale': 4170, 'From Line 1': 4169, 'From Line 2': 4169, 'From Line 3': 4169, 'From Line 4': 4169, 'From Line 5': 4073, 'To Line 1

Number of transfers extracted: 1308
Keys: {'Number': 1308, 'Action': 1308, 'Name': 1308, 'Rationale': 1308, 'From Line 1': 1291, 'From Line 2': 1291, 'From Line 3': 1291, 'From Line 4': 1291, 'From Line 5': 1262, 'To Line 1': 1291, 'To Line 2': 1291, 'To Line 3': 1291, 'To Line 4': 1291, 'To Line 5': 1251, 'Amount': 1308, 'UNFORMATTED': 72} 

Document Name: Board of Education 2018-08-22 - Agenda For The Board Of Education
Document URL: https://www.documentcloud.org/documents/6512038-Board-of-Education-2018-08-22-Agenda-For-The.html
Number of transfers extracted: 637
Keys: {'Number': 637, 'Action': 637, 'Name': 637, 'Rationale': 637, 'From Line 1': 635, 'From Line 2': 635, 'From Line 3': 635, 'From Line 4': 635, 'From Line 5': 597, 'To Line 1': 635, 'To Line 2': 635, 'To Line 3': 635, 'To Line 4': 635, 'To Line 5': 634, 'Amount': 637, 'UNFORMATTED': 78} 

Document Name: Board of Education 2018-04-25 - Agenda For The Board Of Education
Document URL: https://www.documentcloud.org/document